In [19]:
from functions.etl_pbp import extract_files, create_spark_session, get_location_of_ball
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,LongType,IntegerType,FloatType,DateType
from pyspark.sql.functions import *

spark = create_spark_session()

In [2]:
spark

In [3]:
urls_dwd = ['https://github.com/sealneaward/nba-movement-data/raw/master/data/01.01.2016.CHA.at.TOR.7z']
folder_raw = '/workspace/azure_pocket/raw_data' 
folder_tmp_path = "/workspace/azure_pocket/raw_data/tmp" # store json temp

game_files = extract_files(folder_raw=folder_raw,folder_tmp_path=folder_tmp_path,urls=urls_dwd)

game_files

['0021500492.json']

In [4]:
movement = get_location_of_ball(folder_tmp_path=folder_tmp_path,game_files=game_files[0])

In [5]:
spark.sparkContext

<SparkContext master=local[*] appName=PySpark processing NBA data>

In [12]:
teste = movement*10

In [21]:
    
# create our schema to upload data
schema_location_of_ball_and_teams = StructType([ \
    StructField("team_id",LongType(),True),
    StructField("player_id",LongType(),True),
    StructField("location_x",FloatType(),True),
    StructField("location_y",FloatType(),True),
    StructField("location_z", FloatType(),True),
    StructField("game_clock", FloatType(),True),
    StructField("shot_clock", FloatType(),True),
    StructField("period", IntegerType(),True),
    StructField("game_id", StringType(),True),
    StructField("event_id", StringType(),True),
    StructField("game_date", StringType(),True)
    ]
)

teste_df = spark.createDataFrame(teste,schema=schema_location_of_ball_and_teams)

In [23]:
teste_df.show(2)

23/03/17 00:14:18 WARN TaskSetManager: Stage 2 contains a task of very large size (93068 KiB). The maximum recommended task size is 1000 KiB.


23/03/17 00:14:22 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 2 (TID 17): Attempting to kill Python Worker
+----------+---------+----------+----------+----------+----------+----------+------+----------+--------+----------+
|   team_id|player_id|location_x|location_y|location_z|game_clock|shot_clock|period|   game_id|event_id| game_date|
+----------+---------+----------+----------+----------+----------+----------+------+----------+--------+----------+
|        -1|       -1|   23.4254|  45.12734|   3.64299|    711.26|     11.99|     1|0021500492|       1|2016-01-01|
|1610612761|     2449|  19.08811|  13.91147|       0.0|    711.26|     11.99|     1|0021500492|       1|2016-01-01|
+----------+---------+----------+----------+----------+----------+----------+------+----------+--------+----------+
only showing top 2 rows



In [17]:
len(teste),len(movement)

(23253550, 2325355)

In [6]:
movement_df = spark.createDataFrame(movement)

In [7]:
movement_df.show()

23/03/17 00:05:08 WARN TaskSetManager: Stage 0 contains a task of very large size (9309 KiB). The maximum recommended task size is 1000 KiB.


+----------+------+--------+--------+-------+------+-----+---+----------+---+----------+
|        _1|    _2|      _3|      _4|     _5|    _6|   _7| _8|        _9|_10|       _11|
+----------+------+--------+--------+-------+------+-----+---+----------+---+----------+
|        -1|    -1| 23.4254|45.12734|3.64299|711.26|11.99|  1|0021500492|  1|2016-01-01|
|1610612761|  2449|19.08811|13.91147|    0.0|711.26|11.99|  1|0021500492|  1|2016-01-01|
|1610612761|201960|10.11935|13.54703|    0.0|711.26|11.99|  1|0021500492|  1|2016-01-01|
|1610612761|200768|20.81838|44.51006|    0.0|711.26|11.99|  1|0021500492|  1|2016-01-01|
|1610612761|201942|14.16535| 38.7957|    0.0|711.26|11.99|  1|0021500492|  1|2016-01-01|
|1610612761|202685|20.89057|34.20938|    0.0|711.26|11.99|  1|0021500492|  1|2016-01-01|
|1610612766|101107|13.30744|22.19981|    0.0|711.26|11.99|  1|0021500492|  1|2016-01-01|
|1610612766|201587|  9.1404|18.58649|    0.0|711.26|11.99|  1|0021500492|  1|2016-01-01|
|1610612766|202689|19